### Import the necessary libraries

In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical

### Loading the data

In [14]:
# load the data from the pickle file in the data/ directory
df = pd.read_pickle('data/words_df.pkl')

In [19]:
df.ImageData[0].shape

(32, 128, 1)

### Splitting train, validation, and test data

### Neural Networks

#### Starting simple